### Import the required libraries and modules:

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision
import numpy as np

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)
%cd /content/gdrive/MyDrive/medical_image_processing/

Mounted at /content/gdrive
/content/gdrive/MyDrive/medical_image_processing


In [ ]:
from fundal_image.utils import create_dir, get_filenames_sorted
from patches import generate_patches_images_only, reconstruct_patches
from metrics import iou, dice_coef, dice_loss, DiceBCELoss, bce
from metrics import compute_metrics, save_results
from utils import read_image_train, read_mask_train

In [ ]:
image_sizes = {"HRF": (2336, 3504), "drive": (584, 565), "chase_db": (960, 999)}
patch_size = {"HRF": (256, 256)}

dataset_name = "HRF"
patch_size = patch_size[dataset_name]
patch_height = patch_size[0]


In [ ]:
loss_str = "dice_loss"
with_augmentation = True
stride=256
seed = 42
run_id = 2
test_percentage = 0.4

aug_string = ""
if with_augmentation:
    aug_string = "_hvr"
root_path = f"/content/datasets/hrf_{patch_height}_{stride}_{loss_str}{aug_string}_{run_id}/"
images_path = f"{root_path}/images/"
gt_path = f"{root_path}/groundtruth"
model_dir = f"{root_path}/trained_models/"



In [ ]:
root_path
create_dir(root_path)

In [ ]:
num_pixels_to_add = [0, 0]
if (image_sizes["HRF"][0] % patch_size[0]) > 0:
    num_pixels_to_add[0] = ((image_sizes["HRF"][0] // patch_size[0]) + 1) * patch_size[0]
    num_pixels_to_add[1] = ((image_sizes["HRF"][1] // patch_size[1]) + 1) * patch_size[1]

split_name = "split_2"
data_path = root_path
splits =["train","test"]

In [ ]:
create_dir(images_path)
create_dir(gt_path)

# Copy images and ground truth

In [ ]:
!cp /content/gdrive/MyDrive/medical_image_processing/fundal_image/datasets/HRF/images/*.* {root_path}/images
!cp /content/gdrive/MyDrive/medical_image_processing/fundal_image/datasets/HRF/groundtruth/*.* {root_path}/groundtruth/



In [ ]:
!ls {images_path}

01_dr.JPG  03_g.jpg   05_h.jpg	 08_dr.JPG  10_g.jpg   12_h.jpg   15_dr.JPG
01_g.jpg   03_h.jpg   06_dr.JPG  08_g.jpg   10_h.jpg   13_dr.JPG  15_g.jpg
01_h.jpg   04_dr.JPG  06_g.jpg	 08_h.jpg   11_dr.JPG  13_g.jpg   15_h.jpg
02_dr.JPG  04_g.jpg   06_h.jpg	 09_dr.JPG  11_g.jpg   13_h.jpg
02_g.jpg   04_h.jpg   07_dr.JPG  09_g.jpg   11_h.jpg   14_dr.JPG
02_h.jpg   05_dr.JPG  07_g.jpg	 09_h.jpg   12_dr.JPG  14_g.jpg
03_dr.JPG  05_g.jpg   07_h.jpg	 10_dr.JPG  12_g.jpg   14_h.jpg


In [ ]:

images_files, gt_files = get_filenames_sorted(images_path, gt_path)
print(len(images_files), len(gt_files))


45 45


In [ ]:
label = [_xtrain.rsplit("/", 1)[1].rsplit(".", 1)[0].rsplit("_", 1)[1] for _xtrain in images_files] 
print(label)

['dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h']


In [ ]:
# train_test_split
xtrain = []
xtest = []
test_x = None
if test_x is None or len(test_x) == 0:
    print("Creating train test split")
    from sklearn.model_selection import train_test_split
    label = [_xtrain.rsplit("/", 1)[1].rsplit(".", 1)[0].rsplit("_", 1)[1] for _xtrain in images_files] 
    xtrain, xtest, ytrain, ytest = train_test_split(images_files, gt_files, stratify=label,  test_size=test_percentage, random_state=seed)
else:
    # copy if there are any images without data_augmentation
    xtrain = [f for f in images_files if f.rsplit("/", 1)[1].rsplit(".", 1)[0] not in test_x]
    xtest = [f for f in images_files if f.rsplit("/", 1)[1].rsplit(".", 1)[0]  in test_x]

    # copy augmented images
    for f in images_files:
        name = f.rsplit("/", 1)[1].rsplit(".", 1)[0]
        if name.rsplit("_", 1)[1].isnumeric():
            if name.rsplit("_", 1)[0] in test_x:
                xtest.append(f)
            else:
                xtrain.append(f)




Creating train test split


In [ ]:
print(len(xtrain), len(xtest))


27 18


In [ ]:
from albumentations import HorizontalFlip, VerticalFlip, Rotate
def augment_data(images, masks, save_path, augment=True):
    H = 2336
    W = 3504

    for idx, (x, y) in tqdm(enumerate(zip(images, masks)), total=len(images)):
        """ Extracting names """
        name = x.rsplit("/", 1)[1].rsplit(".", 1)[0]
        print(name)
        """ Reading image and mask """
        x = cv2.imread(x, cv2.IMREAD_COLOR)
        y = cv2.imread(y, cv2.IMREAD_GRAYSCALE)

        if augment == True:
            aug = HorizontalFlip(p=1.0)
            augmented = aug(image=x, mask=y)
            x1 = augmented["image"]
            y1 = augmented["mask"]

            aug = VerticalFlip(p=1.0)
            augmented = aug(image=x, mask=y)
            x2 = augmented["image"]
            y2 = augmented["mask"]

            aug = Rotate(limit=45, p=1.0)
            augmented = aug(image=x, mask=y)
            x3 = augmented["image"]
            y3 = augmented["mask"]

            X = [x, x1, x2, x3]
            Y = [y, y1, y2, y3]

            X = [x, x1, x2, x3]
            Y = [y, y1, y2, y3]

        else:
            X = [x]
            Y = [y]

        index = 0
        for i, m in zip(X, Y):
            i = cv2.resize(i, (W, H))
            m = cv2.resize(m, (W, H))

            if len(X) == 1:
                tmp_image_name = f"{name}.png"
                tmp_mask_name = f"{name}.png"
            else:
                tmp_image_name = f"{name}_{index}.png"
                tmp_mask_name = f"{name}_{index}.png"

            image_path = os.path.join(save_path, "image", tmp_image_name)
            mask_path = os.path.join(save_path, "mask", tmp_mask_name)

            cv2.imwrite(image_path, i)
            cv2.imwrite(mask_path, m)

            index += 1

In [ ]:
create_dir(f"{root_path}/augmented/train/image/")
create_dir(f"{root_path}/augmented/train/mask/")

In [ ]:
from tqdm import tqdm

In [ ]:
augment_data(xtrain, ytrain, f"{root_path}/augmented/train/", True )

  0%|          | 0/27 [00:00<?, ?it/s]

13_g


  4%|▎         | 1/27 [00:01<00:49,  1.90s/it]

11_g


  7%|▋         | 2/27 [00:03<00:47,  1.92s/it]

...

 93%|█████████▎| 25/27 [00:45<00:03,  1.82s/it]

05_h


 96%|█████████▋| 26/27 [00:47<00:01,  1.81s/it]

03_dr


100%|██████████| 27/27 [00:49<00:00,  1.83s/it]


In [ ]:
xtrain, ytrain = get_filenames_sorted(f"{root_path}/augmented/train/image", f"{root_path}/augmented/train/mask")

In [ ]:
print(len(xtrain), len(ytrain))

108 108


In [ ]:
split = "train"
#for split in splits:
image_patches_path = f"{data_path}/{split_name}/{split}/image/"
create_dir(image_patches_path)
gt_patches_path = f"{data_path}/{split_name}/{split}/mask/"
create_dir(gt_patches_path)

im_patches_dict_train, metadata_train_df, gt_patches_dict_train = generate_patches_images_only(xtrain,
                                                                image_patches_path,
                                                                patch_size=patch_size,
                                                                stride=stride,
                                                                zero_padding_size=num_pixels_to_add,
                                                                gt_files=ytrain,
                                                                save_path_gt=gt_patches_path)
patch_height = patch_size[0]
metadata_train_df.to_csv(f"{data_path}/{split_name}/image_patches_{patch_height}_{stride}_{split}.csv", index=False)

Generating patches for 108 images


100%|██████████| 108/108 [01:20<00:00,  1.33it/s]


In [ ]:
split = "test"
#for split in splits:
image_patches_path = f"{data_path}/{split_name}/{split}/image/"
create_dir(image_patches_path)
gt_patches_path = f"{data_path}/{split_name}/{split}/mask/"
create_dir(gt_patches_path)
im_patches_dict_test, metadata_test_df, gt_patches_dict_test = generate_patches_images_only(xtest,
                                                                image_patches_path,
                                                                patch_size=patch_size,
                                                                stride=stride,
                                                                zero_padding_size=num_pixels_to_add,
                                                                gt_files=ytest,
                                                                save_path_gt=gt_patches_path)
metadata_test_df.to_csv(f"{data_path}/{split_name}/image_patches_{patch_size}_{stride}_{split}.csv", index=False)


Generating patches for 18 images


100%|██████████| 18/18 [00:11<00:00,  1.52it/s]


In [ ]:
metadata_test_df.shape

(2520, 7)

In [ ]:
metadata_train_df.head()

,image_filename,row,col,patch_height,patch_width,stride,num_fg_pixels
0,/content/datasets/hrf_256_256_dice_loss_hvr_2/...,0,0,256,256,256,17825
1,/content/datasets/hrf_256_256_dice_loss_hvr_2/...,0,256,256,256,256,21296
2,/content/datasets/hrf_256_256_dice_loss_hvr_2/...,0,512,256,256,256,32971
3,/content/datasets/hrf_256_256_dice_loss_hvr_2/...,0,768,256,256,256,36864
4,/content/datasets/hrf_256_256_dice_loss_hvr_2/...,0,1024,256,256,256,36864


In [ ]:
import glob
split = "test"
image_patches_path = f"{data_path}/{split_name}/{split}/image/"

reconstructed_original_path = f"{data_path}/{split_name}/reconstructed/{split}/image"
create_dir(reconstructed_original_path)
reconstruct_patches(im_patches_dict_test,
                    metadata_test_df,
                    reconstructed_original_path
                    )


In [ ]:
reconstructed_gt_path = f"{data_path}/{split_name}/reconstructed/{split}/mask"

create_dir(reconstructed_gt_path)
reconstruct_patches(gt_patches_dict_test,
                    metadata_test_df,
                    reconstructed_gt_path,
                    )


In [ ]:
print([k for k in im_patches_dict_test.keys()])

['11_dr', '05_dr', '07_dr', '01_h', '15_g', '06_h', '15_h', '04_dr', '10_h', '06_g', '02_g', '05_g', '13_dr', '09_h', '03_g', '08_dr', '08_g', '12_h']


### Construct the Model:

In [ ]:
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="UNET")
    return model

input_shape = (patch_size[0], patch_size[1], 3)
model = build_unet(input_shape)
model.summary()

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 256, 256, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                              



### Training the Model:

In [ ]:
if loss_str == "bce":
    loss = bce
elif loss_str == "dice_bce":
    loss = DiceBCELoss
elif loss_str == "dice_loss":
    loss = dice_loss
else:
    raise Exception("Invalid value for loss_str")
    


In [ ]:
loss

<function metrics.dice_loss>

In [ ]:
H = patch_size[0]
W = patch_size[1]

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_data(path):
    x = sorted(glob(os.path.join(path, "image", "*.png")))
    y = sorted(glob(os.path.join(path, "mask", "*.png")))
    return x, y

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image_train(x)
        y = read_mask_train(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(X, Y, batch_size=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(4)
    return dataset

In [ ]:
from keras.metrics import AUC

In [ ]:
create_dir(model_dir)
%cd /content/datasets

/content/datasets


In [ ]:
num_epochs_completed = 0

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import CustomObjectScope


In [ ]:
from glob import glob

In [ ]:
""" Seeding """
np.random.seed(42)
tf.random.set_seed(42)

""" Directory to save files """
create_dir(model_dir)

""" Hyperparameters """
batch_size = 16
lr = 1e-4
num_epochs = 50

last_model_path = os.path.join(f"{model_dir}", f"model_{num_epochs_completed}.h5")

model_path = os.path.join(f"{model_dir}", "model.h5")
csv_path = os.path.join(f"{model_dir}", "data.csv")

""" Dataset """
dataset_path = f"{data_path}/{split_name}"
train_path = os.path.join(dataset_path, "train")
valid_path = os.path.join(dataset_path, "test")

train_x, train_y = load_data(train_path)
train_x, train_y = shuffling(train_x, train_y)
valid_x, valid_y = load_data(valid_path)

print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Valid: {len(valid_x)} - {len(valid_y)}")

train_dataset = tf_dataset(train_x, train_y, batch_size=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch_size=batch_size)

train_steps = len(train_x)//batch_size
valid_setps = len(valid_x)//batch_size

if len(train_x) % batch_size != 0:
    train_steps += 1
if len(valid_x) % batch_size != 0:
    valid_setps += 1

""" Model """
model = build_unet((H, W, 3))
# with CustomObjectScope({"DiceBCELoss":DiceBCELoss, "dice_coef":dice_coef, "iou":iou}):
#     model = load_model(last_model_path)
model.compile(loss=loss, optimizer=Adam(lr), metrics=[dice_coef, iou, Recall(), Precision(), AUC()])

callbacks = [
    ModelCheckpoint(model_path, verbose=1, save_best_only=True, monitor="val_dice_coef", mode="max"),
    ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, min_lr=1e-6, verbose=1),
    CSVLogger(csv_path),
    TensorBoard(),
    EarlyStopping(monitor="val_dice_coef", patience=10, restore_best_weights=False, mode="max")
]

model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=valid_dataset,
    steps_per_epoch=train_steps,
    validation_steps=valid_setps,
    callbacks=callbacks
)

Train: 15120 - 15120
Valid: 2520 - 2520
Epoch 1/50
945/945 [==============================] - ETA: 0s - loss: 0.3566 - dice_coef: 0.6434 - iou: 0.4854 - recall: 0.8078 - precision: 0.6340 - auc: 0.9321
Epoch 00001: val_dice_coef improved from -inf to 0.73677, saving model to /content/datasets/hrf_256_256_dice_loss_hvr_2//trained_models/model.h5
945/945 [==============================] - 417s 424ms/step - loss: 0.3566 - dice_coef: 0.6434 - iou: 0.4854 - recall: 0.8078 - precision: 0.6340 - auc: 0.9321 - val_loss: 0.2633 - val_dice_coef: 0.7368 - val_iou: 0.5879 - val_recall: 0.8212 - val_precision: 0.7754 - val_auc: 0.9377 - lr: 1.0000e-04
Epoch 2/50
945/945 [==============================] - ETA: 0s - loss: 0.2263 - dice_coef: 0.7737 - iou: 0.6324 - recall: 0.8105 - precision: 0.7887 - auc: 0.9395
Epoch 00002: val_dice_coef improved from 0.73677 to 0.78895, saving model to /content/datasets/hrf_256_256_dice_loss_hvr_2//trained_models/model.h5
945/945 [==============================] - 

# Evaluating the Model:

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import pandas as pd
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score
from tensorflow.keras.models import load_model

In [ ]:
zip_filename = root_path.rsplit("/")[-2]
zip_filename += ".zip"

In [ ]:
!ls /content/gdrive/MyDrive/Blood_Vessel_Results/hrf_patches/hrf_256_*

/content/gdrive/MyDrive/Blood_Vessel_Results/hrf_patches/hrf_256_256_dice_loss_1.zip
/content/gdrive/MyDrive/Blood_Vessel_Results/hrf_patches/hrf_256_256_dice_loss_hvr_1.zip

/content/gdrive/MyDrive/Blood_Vessel_Results/hrf_patches/hrf_256_256_dice_loss_1:
groundtruth  predictions     results		    split_2
images	     predictions_np  results_reconstructed  trained_models


In [ ]:
%cd /content/datasets
!cp /content/gdrive/MyDrive/Blood_Vessel_Results/hrf_patches/{zip_filename} .

/content/datasets
cp: cannot stat '/content/gdrive/MyDrive/Blood_Vessel_Results/hrf_patches/hrf_256_256_dice_loss_hvr_2.zip': No such file or directory


In [ ]:
!ls

hrf_256_256_dice_loss_hvr_1  hrf_256_256_dice_loss_hvr_2  logs


In [ ]:
!unzip {zip_filename}

unzip:  cannot find or open hrf_256_256_dice_loss_hvr_2.zip, hrf_256_256_dice_loss_hvr_2.zip.zip or hrf_256_256_dice_loss_hvr_2.zip.ZIP.


In [ ]:
H = patch_size[0]
W = patch_size[1]

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.float32)
    return ori_x, x

def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.int32)
    return ori_x, x

def load_data(path):
    x = sorted(glob(os.path.join(path, "image", "*.png")))
    y = sorted(glob(os.path.join(path, "mask", "*.png")))
    return x, y

def save_results(ori_x, ori_y, y_pred, save_image_path):
    line = np.ones((H, 10, 3)) * 255

    ori_y = np.expand_dims(ori_y, axis=-1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) * 255

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred], axis=1)
    cv2.imwrite(save_image_path, cat_images)

In [ ]:
import glob
supported_extensions = ["png", "jpg", "tif", "jpeg", "gif", "bmp"]

def image_files(path, file_prefix=None):
    all_image_files = []
    # print("path", path)
    # print("file_prefix", file_prefix)
    if file_prefix is None:
        file_names = list(glob.glob(path + "/*.*" ))
    else:
        prefix___ = path + "/" + file_prefix + ".*"
        #print("prefix", prefix___)
        file_names = list(glob.glob(prefix___))

    # print("filenames", file_names)

    for filename in file_names:
        ext = filename.rsplit(".", 1)[1]
        if ext.lower() in supported_extensions:
            all_image_files.append(filename)
    # print("All Image files", all_image_files)
    return sorted(all_image_files)


In [ ]:
results_path = f"{root_path}/results/"
prediction_path = f"{root_path}/predictions/"
prediction_np_path = f"{root_path}/predictions_np/"
create_dir(prediction_path)
create_dir(prediction_np_path)
create_dir(results_path)

In [ ]:
results_path

'/content/datasets/hrf_256_256_dice_loss_hvr_2//results/'

In [ ]:
from glob import glob

In [ ]:
model_path = os.path.join(f"{model_dir}", "model.h5")
csv_path = os.path.join(f"{model_dir}", "data.csv")

In [ ]:
model = load_model(model_path, compile=False)

In [ ]:
split="test"
dataset_path = f"{root_path}/{split_name}/{split}/"
df, score = compute_metrics(dataset_path,
                    results_path=results_path,
                    threshold=0.5,
                    model=model,
                    predictions_path=prediction_path,
                    prediction_np_path=prediction_np_path)

Threshold 0.5


  0%|          | 0/2520 [00:00<?, ?it/s]

_01_h_0


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Prec

(256, 256, 1) 9.172952e-06 3.1133467e-07
(256, 256, 3) (256, 256) (256, 256)
_01_h_1
(256, 256, 1) 9.502838e-06 2.2953886e-07
(256, 256, 3) (256, 256) (256, 256)


...

 19%|█▉        | 475/2520 [01:19<05:41,  5.99it/s]

_04_dr_21
(256, 256, 1) 1.0 6.2706936e-13
(256, 256, 3) (256, 256) (256, 256)
_04_dr_22


 19%|█▉        | 476/2520 [01:19<05:49,  5.85it/s]

(256, 256, 1) 1.0 4.300994e-13
(256, 256, 3) (256, 256) (256, 256)
_04_dr_23
(256, 256, 1) 1.0 1.4347239e-13
(256, 256, 3) (256, 256) (256, 256)


 19%|█▉        | 477/2520 [01:20<05:50,  5.83it/s]

In [ ]:
model_dir

# Reconstruct predicted image

In [ ]:
prediction_np_path

In [ ]:
from glob import glob

In [ ]:
# Create patches dictionary
predicted_patches_dict = dict()
for file in glob(f"{prediction_np_path}/*.npy"):
    print("file", file)
    patch = np.load(file)
    fname_without_ext = file.rsplit("/", 1)[1].rsplit(".", 1)[0]
    fname_without_ext = fname_without_ext[1:]
    patch_number = int(fname_without_ext.rsplit("_", 1)[1])
    image_file = fname_without_ext.rsplit("_", 1)[0]
    print(image_file, patch_number )

    if image_file in predicted_patches_dict.keys():
        predicted_patches_dict[image_file][patch_number] = np.load(file)
    else:
        patches = [None] * 140
        patches[patch_number] = np.load(file)
        predicted_patches_dict[image_file] = patches

        




In [ ]:
reconstructed_predictions_path = f"{data_path}/{split_name}/reconstructed/{split}/predictions_reconstructed"
create_dir(reconstructed_predictions_path)
reconstruct_patches(predicted_patches_dict,
                    metadata_test_df,
                    reconstructed_predictions_path,
                    mfactor=255
                    )


In [ ]:
!pwd

In [ ]:
def read_mask(path):
    print(path)
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
#    x = x.astype(np.int32)
    return ori_x, x

In [ ]:
def save_results(ori_x, ori_y, y_pred, save_image_path):
    line = np.ones((2560, 10, 3)) * 255

    ori_y = np.expand_dims(ori_y, axis=-1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) * 255

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred], axis=1)
    print(f"Saving image of size {cat_images.shape} to {save_image_path}")
    cv2.imwrite(save_image_path, cat_images)

In [ ]:
results_path_reconstructed = f"{root_path}/results_reconstructed/"
create_dir(results_path_reconstructed)

In [ ]:
from glob import glob

In [ ]:
dataset_path = f"{root_path}/{split_name}/reconstructed/test/"

In [ ]:
df, scores = compute_metrics(dataset_path,
                    results_path=results_path_reconstructed,
                    threshold=0.4,
                    model=None,
                    predictions_path=f"{root_path}/{split_name}/reconstructed/test/predictions_reconstructed/",
                    prediction_np_path=None)

Threshold 0.4


  0%|          | 0/18 [00:00<?, ?it/s]

01_h
Reading predicted image from  /content/datasets/hrf_256_256_dice_loss_hvr_2//split_2/reconstructed/test/predictions_reconstructed//01_h.png
(2560, 3584) 255 0
(2560, 3584) 1 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)


  6%|▌         | 1/18 [00:15<04:23, 15.49s/it]

02_g
Reading predicted image from  /content/datasets/hrf_256_256_dice_loss_hvr_2//split_2/reconstructed/test/predictions_reconstructed//02_g.png
(2560, 3584) 255 0
(2560, 3584) 1 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)


...

15_g
Reading predicted image from  /content/datasets/hrf_256_256_dice_loss_hvr_2//split_2/reconstructed/test/predictions_reconstructed//15_g.png
(2560, 3584) 255 0
(2560, 3584) 1 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)


 94%|█████████▍| 17/18 [04:23<00:15, 15.54s/it]

15_h
Reading predicted image from  /content/datasets/hrf_256_256_dice_loss_hvr_2//split_2/reconstructed/test/predictions_reconstructed//15_h.png
(2560, 3584) 255 0
(2560, 3584) 1 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)


100%|██████████| 18/18 [04:39<00:00, 15.53s/it]

Threshold: 0.40000
Accuracy: 0.97530
F1: 0.81656
Jaccard: 0.69135
Recall: 0.82141
Precision: 0.81830
AUC: 0.90400
Dice Coeff: 0.81656


In [ ]:
df, scores = compute_metrics(dataset_path,
                    results_path=results_path_reconstructed,
                    threshold=0.5,
                    model=None,
                    predictions_path=f"{root_path}/{split_name}/reconstructed/test/predictions_reconstructed/",
                    prediction_np_path=None)

Threshold 0.5


  0%|          | 0/18 [00:00<?, ?it/s]

01_h
Reading predicted image from  /content/datasets/hrf_256_256_dice_loss_hvr_2//split_2/reconstructed/test/predictions_reconstructed//01_h.png
(2560, 3584) 255 0
(2560, 3584) 1 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)


  6%|▌         | 1/18 [00:15<04:24, 15.53s/it]

02_g
Reading predicted image from  /content/datasets/hrf_256_256_dice_loss_hvr_2//split_2/reconstructed/test/predictions_reconstructed//02_g.png
(2560, 3584) 255 0
(2560, 3584) 1 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)


...

15_g
Reading predicted image from  /content/datasets/hrf_256_256_dice_loss_hvr_2//split_2/reconstructed/test/predictions_reconstructed//15_g.png
(2560, 3584) 255 0
(2560, 3584) 1 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)


 94%|█████████▍| 17/18 [04:24<00:15, 15.55s/it]

15_h
Reading predicted image from  /content/datasets/hrf_256_256_dice_loss_hvr_2//split_2/reconstructed/test/predictions_reconstructed//15_h.png
(2560, 3584) 255 0
(2560, 3584) 1 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)


100%|██████████| 18/18 [04:40<00:00, 15.59s/it]

Threshold: 0.50000
Accuracy: 0.97530
F1: 0.81656
Jaccard: 0.69135
Recall: 0.82141
Precision: 0.81830
AUC: 0.90400
Dice Coeff: 0.81656


In [ ]:
df, scores = compute_metrics(dataset_path,
                    results_path=results_path_reconstructed,
                    threshold=0.6,
                    model=None,
                    predictions_path=f"{root_path}/{split_name}/reconstructed/test/predictions_reconstructed/",
                    prediction_np_path=None)

Threshold 0.6


  0%|          | 0/18 [00:00<?, ?it/s]

01_h
Reading predicted image from  /content/datasets/hrf_256_256_dice_loss_hvr_2//split_2/reconstructed/test/predictions_reconstructed//01_h.png
(2560, 3584) 255 0
(2560, 3584) 1 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)


  6%|▌         | 1/18 [00:15<04:22, 15.47s/it]

02_g
Reading predicted image from  /content/datasets/hrf_256_256_dice_loss_hvr_2//split_2/reconstructed/test/predictions_reconstructed//02_g.png
(2560, 3584) 255 0
(2560, 3584) 1 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)


...

15_g
Reading predicted image from  /content/datasets/hrf_256_256_dice_loss_hvr_2//split_2/reconstructed/test/predictions_reconstructed//15_g.png
(2560, 3584) 255 0
(2560, 3584) 1 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)


 94%|█████████▍| 17/18 [04:23<00:15, 15.50s/it]

15_h
Reading predicted image from  /content/datasets/hrf_256_256_dice_loss_hvr_2//split_2/reconstructed/test/predictions_reconstructed//15_h.png
(2560, 3584) 255 0
(2560, 3584) 1 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)


100%|██████████| 18/18 [04:38<00:00, 15.49s/it]

Threshold: 0.60000
Accuracy: 0.97530
F1: 0.81656
Jaccard: 0.69135
Recall: 0.82141
Precision: 0.81830
AUC: 0.90400
Dice Coeff: 0.81656


In [ ]:
history_df = pd.read_csv(csv_path)

In [ ]:
history_df.shape

(25, 14)

In [ ]:
history_df.head(50)

,epoch,auc,dice_coef,iou,loss,lr,precision,recall,val_auc,val_dice_coef,val_iou,val_loss,val_precision,val_recall
0,0,0.932098,0.643424,0.485427,0.356577,0.00010,0.633961,0.807798,0.937724,0.736774,0.587943,0.263271,0.775385,0.821211
1,1,0.939542,0.773744,0.632393,0.226256,0.00010,0.788685,0.810547,0.936253,0.788950,0.654557,0.211199,0.814901,0.810783
2,2,0.928829,0.794750,0.660518,0.205250,0.00010,0.799214,0.812636,0.922630,0.799243,0.668378,0.200908,0.822751,0.806280
3,3,0.922771,0.804318,0.673666,0.195681,0.00010,0.804920,0.817154,0.926746,0.802919,0.673669,0.197201,0.796494,0.833909
4,4,0.919928,0.809407,0.680753,0.190593,0.00010,0.808296,0.820197,0.917642,0.806840,0.678938,0.193288,0.814491,0.819646
5,5,0.918543,0.812797,0.685519,0.187203,0.00010,0.810946,0.822527,0.920611,0.809694,0.682942,0.190443,0.807130,0.831088
6,6,0.917775,0.815634,0.689533,0.184366,0.00010,0.813053,0.825132,0.916123,0.799970,0.669467,0.200168,0.793176,0.827143
7,7,0.917345,0.817483,0.692145,0.182516,0.00010,0.814624,0.826697,0.919818,0.807031,0.679341,0.193067,0.799559,0.834120
8,8,0.917332,0.819746,0.695363,0.180254,0.00010,0.816948,0.828517,0.909179,0.804792,0.676229,0.195307,0.817071,0.812373
9,9,0.917441,0.821708,0.698174,0.178292,0.00010,0.818599,0.830501,0.917588,0.808640,0.681541,0.191450,0.800745,0.834921


In [ ]:
results_path

'/content/datasets/hrf_256_256_dice_loss_hvr_2//results/'

In [ ]:
!zip -r hrf_256_256_dice_loss_hvr_2.zip  hrf_256_256_dice_loss_hvr_2/

Streaming output truncated to the last 5000 lines.
  adding: hrf_256_256_dice_loss_hvr_2/split_2/train/mask/_11_h_1_5.png (deflated 32%)
  adding: hrf_256_256_dice_loss_hvr_2/split_2/train/mask/_13_h_1_109.png (deflated 25%)
  adding: hrf_256_256_dice_loss_hvr_2/split_2/train/mask/_07_g_0_79.png (deflated 24%)
  adding: hrf_256_256_dice_loss_hvr_2/split_2/train/mask/_14_g_3_78.png (deflated 23%)
  adding: hrf_256_256_dice_loss_hvr_2/split_2/train/mask/_10_dr_3_46.png (deflated 26%)
  adding: hrf_256_256_dice_loss_hvr_2/split_2/train/mask/_01_dr_1_34.png (deflated 23%)
  adding: hrf_256_256_dice_loss_hvr_2/split_2/train/mask/_12_g_3_38.png (deflated 23%)
  adding: hrf_256_256_dice_loss_hvr_2/split_2/train/mask/_06_dr_0_14.png (deflated 85%)
  adding: hrf_256_256_dice_loss_hvr_2/split_2/train/mask/_12_dr_0_53.png (deflated 27%)
  adding: hrf_256_256_dice_loss_hvr_2/split_2/train/mask/_14_h_1_13.png (deflated 85%)
  adding: hrf_256_256_dice_loss_hvr_2/split_2/train/mask/_03_h_2_36.png (de

In [ ]:
!cp hrf_256_256_dice_loss_hvr_2.zip /content/gdrive/MyDrive/Blood_Vessel_Results/hrf_patches/

In [ ]:
drive.flush_and_unmount()